In [1]:
%run ./bonus_assignment.ipynb

@register_cell_magic
def fakedata_testing(line, cell):
    # wrapping everything inside a giant try-except block
    try:
        fakedata(line,cell)
        print("No exceptions encountered while generating DataFrames 🐼🎉🎈🎊")
    except Exception as e:
        template = "🚫🚫🚫🚫🚫 An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(e).__name__, e.args)
        print(message)

@register_line_magic
def test(line):
    try:
        exec(line)
        print("PASSED")
    except Exception as e:
        print("FAILED {}".format(e))

In [2]:
%%fakedata_testing
persons
-------
first_name
last_name*
phone_number
random_number(5) as customer_number [1]*

purchases
---------
isbn10
credit_card_full
random_number(3) as price
random_number(5) as customer_number [1]

No exceptions encountered while generating DataFrames 🐼🎉🎈🎊


In [3]:
%test assert len(persons) == 99,'"persons" should have 99 rows'
%test assert len(purchases) == 99,'"purchases" should have 99 rows'
%test assert len(persons.last_name.value_counts()) == 99,'"persons.last_name" should have 99 unique values'
%test assert len(persons.customer_number.value_counts()) == 99,'"persons.customer_number" should have 99 unique values'
%test assert len(purchases.customer_number.value_counts()) < 99,'"purchases.customer_numbers" should have less than 99 unique values'
%test assert purchases.customer_number.isin(persons.customer_number).all(), '"purchases.customer_number" should be a subset of "persons.customer_number"'
%test assert (purchases.columns == ['isbn10', 'credit_card_full', 'price', 'customer_number']).all(), '''"purchase.columns" should be ['isbn10', 'credit_card_full', 'price', 'customer_number']'''

PASSED
PASSED
PASSED
PASSED
PASSED
PASSED
PASSED


In [4]:
%%fakedata_testing
companies
-------
company*
catch_phrase
country
date_time_this_century as established

No exceptions encountered while generating DataFrames 🐼🎉🎈🎊


In [5]:
%test assert len(companies) == 99,'"companies" should have 99 rows'
%test assert len(companies.company.value_counts()) == 99,'"companies.company" should have 99 unique values'
%test assert (companies.columns == ['company', 'catch_phrase', 'country', 'established']).all(), '''"purchase.columns" should be ['company', 'catch_phrase', 'country', 'established']'''

PASSED
PASSED
PASSED


---

I added `df_size` support

```
function_to_call  ::= <wordcharacters>
parameters        ::= "" | "(" ( wordcharacters | number ) ")"
as_name           ::= "" | "as" <whitespace> <wordcharacters>
column_name       ::= as_name | function_to_call
reference         ::= "" |  "[" number "]"
unique_mark       ::= "" | "*"
column_definition ::= <function_to_call> <parameters> <whitespace> \
                      <as_name> <whitespace> <reference> <unique_mark>
df_size           ::= "" |  "[" integer "]"
df_sep            ::= "--" ("-"*)
df_definition     ::= <wordcharacters> <df_size> <newline> <df_sep> <newline> \
                      (<column_definition>*) <newline> <newline>
language_spec     ::= <def_definition>*
```
---
Feel free to play around with these as well:
```
DEFAULT_DF_SIZE = 99     # default number of rows per DataFrame
ORPHANED_UNIQUES = 0.2   # % of uniques references which won't be used in non-unique references
REPEAT_WEIGHTS = 1000    # Weight of chance for repeated values.  1 would be equal weights
```

In [6]:
DEFAULT_DF_SIZE = 20     # default number of rows per DataFrame
ORPHANED_UNIQUES = 0.1   # % of uniques references which won't be used in non-unique references
REPEAT_WEIGHTS = 1000    # Weight of chance for repeated values.  1 would be equal weights

In [7]:
%%fakedata_testing
apersons
-------
first_name
last_name*
phone_number
random_number(5) as customer_number [1]*

apurchases [500]
---------
isbn10
credit_card_full
random_number(3) as price
random_number(5) as customer_number [1]

No exceptions encountered while generating DataFrames 🐼🎉🎈🎊


In [8]:
%test assert len(apersons) == DEFAULT_DF_SIZE,'"apersons" should have {} rows'.format(DEFAULT_DF_SIZE)
%test assert len(apurchases) == 500,'"apurchases" should have 500 rows'
%test assert len(apersons.last_name.value_counts()) == DEFAULT_DF_SIZE,'"apersons.last_name" should have {} unique values'.format(DEFAULT_DF_SIZE)
%test assert len(apersons.customer_number.value_counts()) == DEFAULT_DF_SIZE,'"apersons.customer_number" should have {} unique values'.format(DEFAULT_DF_SIZE)
%test assert apurchases.customer_number.isin(apersons.customer_number).all(), '"apurchases.customer_number" should be a subset of "persons.customer_number"'
%test assert (apurchases.columns == ['isbn10', 'credit_card_full', 'price', 'customer_number']).all(), '''"apurchase.columns" should be ['isbn10', 'credit_card_full', 'price', 'customer_number']'''

PASSED
PASSED
PASSED
PASSED
PASSED
PASSED


In [9]:
# distribution of customer_number for 'apurchases' DataFrame
# tweak the 3 options above and rerun the cells to see how they affect the results
apurchases.customer_number.value_counts()

14193    59
57949    54
42039    49
36791    45
36280    42
5817     42
55098    41
6243     39
3680     32
39814    21
25397    21
75302    12
11200    12
243      11
89541     8
33590     6
94752     4
65762     2
Name: customer_number, dtype: int64

In [10]:
%%fakedata_testing
trying_too_many_uniques [1001]
-------
last_name*

🚫🚫🚫🚫🚫 An exception of type AssertionError occurred. Arguments:
('Insufficient data (1000 available) from provider to generate 1001 "last_name".',)


Also support `list` and `dict` for parameters
------------

In [11]:
import datetime
# importing datetime otherwise can't use datetime constructors

In [12]:
%%fakedata
multi_param [20]
-------
color(['red', 'dark'])
color([(300, 20), 'random', 'hsl']) as color2
date(%Y-%m-%d)
date(['%Y-%m-%d', datetime.datetime.strptime('12/11/1971 09:15:32', '%d/%m/%Y %H:%M:%S')]) as date2
date_between({'start_date':'-5d', 'end_date':'-1d'})
date_of_birth([None, 30, 50])
date_this_decade([True, True])
file_path([2, None, 'mp3'])
paragraph([1, True])

In [13]:
multi_param.head()
# no time to create test cases :(

,color,color2,date,date2,date_between,date_of_birth,date_this_decade,file_path,paragraph
0,#ad1f0f,"hsl(40, 35, 50)",2007-02-12,1971-02-07,2020-09-23,1972-07-11,2024-09-14,/family/people/know.mp3,Central road describe.
1,#aa2301,"hsl(250, 86, 17)",1972-01-11,1970-02-04,2020-09-23,1989-12-03,2026-04-02,/loss/officer/truth.mp3,Social drug exist explain begin no.
2,#87060b,"hsl(86, 78, 25)",1991-11-30,1971-09-19,2020-09-24,1984-09-27,2029-09-11,/opportunity/quite/lot.mp3,Matter still even have police down.
3,#b71c0b,"hsl(65, 80, 5)",2008-01-30,1970-12-22,2020-09-25,1975-10-29,2021-05-19,/movie/factor/early.mp3,Rule sport economy sign table.
4,#a32600,"hsl(80, 81, 7)",1997-11-13,1970-09-01,2020-09-24,1973-08-17,2024-06-02,/reality/final/he.mp3,Program story claim phone.
